<a href="https://colab.research.google.com/github/diogopaz/projeto-marvel/blob/DatabaseBrch/projeto_marvel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando bibliotecas necessárias

In [4]:
!pip install dotenv

In [5]:
import requests
import hashlib
import time
import os
import pandas as pd
from dotenv import load_dotenv
import sqlite3
from google.colab import userdata

# Carregando variáveis de ambiente
Estamos fazendo upload do arquivo .env com as chaves de acesso à API no ambeinte de execução do colab.

In [ ]:
load_dotenv(dotenv_path='/content/.env')
public_key = os.getenv('MARVEL_PUBLIC_KEY')
private_key = os.getenv('MARVEL_PRIVATE_KEY')

# Montando os parâmetros para realizar a requisição

In [ ]:
ts = str(time.time())
to_hash = ts + str(private_key) + str(public_key)
hash_md5 = hashlib.md5(to_hash.encode('utf-8')).hexdigest()

params = {
    'apikey': public_key,
    'ts': ts,
    'hash': hash_md5,
    'limit': 100
}

# Realizando a requisição

In [ ]:
url = 'https://gateway.marvel.com/v1/public/characters'

response = requests.get(url, params=params)
data = response.json()
data

In [ ]:
df = pd.DataFrame(data['data']['results'])
df.columns

# Criação da tabela de personagens no banco de dados sqlite

In [ ]:
url = 'https://gateway.marvel.com/v1/public/characters'
# >>> Conecta (ou cria) o banco SQLite
conn = sqlite3.connect('marvel.db')
cursor = conn.cursor()

# >>> Cria tabela characters (sem thumbnail/resource_uri)
cursor.execute('''
CREATE TABLE IF NOT EXISTS characters (
    id INTEGER PRIMARY KEY,
    name TEXT,
    description TEXT,
    modified TEXT
)
''')

# >>> Cria tabela character_comics (relacionamento N:N entre personagens e comics)
cursor.execute('''
CREATE TABLE IF NOT EXISTS character_comics (
    character_id INTEGER,
    comic_id INTEGER,
    PRIMARY KEY (character_id, comic_id)
)
''')

conn.commit()

# >>> Função para montar hash exigido pela Marvel
def get_auth_params():
    ts = str(time.time())
    to_hash = ts + private_key + public_key
    hash_md5 = hashlib.md5(to_hash.encode()).hexdigest()
    return {
        'ts': ts,
        'apikey': public_key,
        'hash': hash_md5
    }

# >>> Função para salvar um personagem no banco
def save_character(char):
    cursor.execute('''
    INSERT OR REPLACE INTO characters (id, name, description, modified)
    VALUES (?, ?, ?, ?)
    ''', (
        char['id'],
        char['name'],
        char.get('description', ''),
        char.get('modified', '')
    ))

    # >>> Salva os comics vinculados ao personagem
    for comic in char.get('comics', {}).get('items', []):
        resource_uri = comic.get('resourceURI', '')
        try:
            comic_id = int(resource_uri.strip().split('/')[-1])
            cursor.execute('''
                INSERT OR IGNORE INTO character_comics (character_id, comic_id)
                VALUES (?, ?)
            ''', (char['id'], comic_id))
        except (IndexError, ValueError):
            print(f'❌ Erro ao extrair comic_id de {resource_uri}')

    conn.commit()

# >>> Baixa e salva todos os personagens
def fetch_and_save_characters():
    limit = 100
    offset = 0
    total = 1  # valor fictício inicial

    while offset < total:
        print(f'⏳ Buscando personagens... offset={offset}')
        params = get_auth_params()
        params.update({
            'limit': limit,
            'offset': offset
        })

        response = requests.get(url, params=params)
        if response.status_code != 200:
            print(f'⚠️ Erro na requisição: {response.status_code}')
            break

        data = response.json()['data']
        total = data['total']
        results = data['results']

        for char in results:
            save_character(char)

        offset += limit
        time.sleep(0.1)  # respeita limites

    print('✅ Todos os personagens e comics foram salvos no banco!')

# >>> Executa
fetch_and_save_characters()

# >>> Fecha conexão
conn.close()



In [7]:
conn = sqlite3.connect('marvel.db')
cursor = conn.cursor()
cursor.execute('select * from characters')
results_sqlite = cursor.fetchall()
conn.close()
results_sqlite

[(1009144,
  'A.I.M.',
  'AIM is a terrorist organization bent on destroying the world.',
  '2013-10-17T14:41:30+0000'),
 (1009146,
  'Abomination (Emil Blonsky)',
  'Formerly known as Emil Blonsky, a spy of Soviet Yugoslavian origin working for the KGB, the Abomination gained his powers after receiving a dose of gamma radiation similar to that which transformed Bruce Banner into the incredible Hulk.',
  '2014-06-27T19:39:07+0000'),
 (1009148, 'Absorbing Man', '', '2016-05-25T12:04:23+0000'),
 (1009149, 'Abyss', '', '2014-04-29T14:10:43+0000'),
 (1009150, 'Agent Zero', '', '1970-01-01T00:00:00+0000'),
 (1009151, 'Amiko', '', '1970-01-01T00:00:00+0000'),
 (1009152, 'Ancient One', '', '2016-08-16T15:14:52+0000'),
 (1009153, 'Angel (Warren Worthington III)', '', '2016-03-07T09:41:51+0000'),
 (1009154, 'Annihilus', '', '2013-11-20T17:06:36+0000'),
 (1009155, 'Ant-Man (Hank Pym)', '', '2021-08-05T15:11:25+0000'),
 (1009156, 'Apocalypse', '', '2014-05-28T12:41:41+0000'),
 (1009157, 'Spider-G